# FIFA API: World Cup teams

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import requests
import json

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [5]:
today_str = pd.Timestamp("today").strftime("%m/%d/%Y")
today = pd.Timestamp("today")

---

## Get data

#### Read teams json from FIFA API

In [6]:
team_json = requests.get(
    "https://api.fifa.com/api/v3/competitions/teams/255711?language=en"
).json()["Results"]

#### Read into a dataframe and lowercase column names

In [7]:
teams_df = pd.DataFrame(team_json)

In [8]:
teams_df.columns = teams_df.columns.str.lower()

#### Deal with nested info

In [9]:
teams_df["team_name"] = pd.json_normalize(pd.json_normalize(teams_df["name"])[0])[
    "Description"
]

In [10]:
teams_df["team_abbr"] = pd.json_normalize(
    pd.json_normalize(teams_df["displayname"])[0]
)["Description"]

In [11]:
teams_df_slim = teams_df[
    [
        "idteam",
        "idconfederation",
        "shortclubname",
        "abbreviation",
    ]
].copy()

In [12]:
teams_df_slim["pictureurl"] = (
    "https://cloudinary.fifa.com/api/v3/picture/flags-sq-4/" + teams_df["abbreviation"]
)

In [13]:
teams_list = teams_df_slim["idteam"].to_list()

---

## Exports

In [14]:
teams_df_slim.to_csv("data/processed/world_cup_teams.csv", index=False)